In [3]:
import numpy  as np
class Variable:
    def __init__(self,data):
        self.data = data

class Function:
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self,x):
        raise NotImplementedError()
    #抛出异常，告诉用户square等函数都是
    #集成于Function，需要自己实现

In [4]:
class Square(Function):
    def forward(self, x):
        return x**2
    
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [5]:
# 利用中心差分近似的数值分析方法来计算导数
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data-eps)
    x1 = Variable(x.data+eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data-y0.data)/(2*eps)

In [6]:
# 计算2.0处square 的导数
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [7]:
# 复合函数可以通过写函数实现，
# 实际上数值分析一般用于梯度检验反向传播是否错误
def f(x):
    A =Square()
    B =Exp()
    C = Square()
    return(C(B(A(x))))

x = Variable(np.array(0.5))
print(numerical_diff(f,x))

3.2974426293330694


In [ ]:
# 反向传播的意义在于只要传播一次，就能求出
# 所有参数的导数
# 但是如果想要进行反向传播，就先要进行一次
# 正向传播 ，存储各函数的输入的变量值

![](../images/backward_trans.png)

In [ ]:
# 扩展variable 记录Grad
class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None